In [83]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import sklearn.neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from sklearn.preprocessing import scale
from collections import Counter

In [105]:
headers = ["carat",	"cut","color","clarity","depth","table","price","x","y","z"]
data = pd.read_csv('diamonds.csv', na_values='?',    
         header=None,  names = headers) 
data = data.reset_index(drop=True)
data = data.iloc[1:]
data.describe()

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940,53940,53940,53940,53940,53940,53940,53940,53940,53940
unique,273,5,7,8,184,127,11602,554,552,375
top,0.3,Ideal,G,SI1,62,56,605,4.37,4.34,2.7
freq,2604,21551,11292,13065,2239,9881,132,448,437,767


In [106]:
#encode
map_cut = {"Fair":1, "Good":2, "Very Good" : 3, "Premium" : 4, "Ideal" : 5}
map_color = {"J": 1,"I":2, "H":3, "G":4, "F":5, "E":6, "D":7}
map_clarity = {"I1":1, "SI2":2, "SI1":3, "VS2":4, "VS1":5, "VVS2":6, "VVS1":7, "IF":8}

data["map_cut"] = data.cut.map(map_cut)
data["map_color"] = data.color.map(map_color)
data["map_clarity"] = data.clarity.map(map_clarity)

data.head()
# data = data[:10]

,carat,cut,color,clarity,depth,table,price,x,y,z,map_cut,map_color,map_clarity
1,0.23,Ideal,E,SI2,61.5,55,326,3.95,3.98,2.43,5,6,2
2,0.21,Premium,E,SI1,59.8,61,326,3.89,3.84,2.31,4,6,3
3,0.23,Good,E,VS1,56.9,65,327,4.05,4.07,2.31,2,6,5
4,0.29,Premium,I,VS2,62.4,58,334,4.2,4.23,2.63,4,2,4
5,0.31,Good,J,SI2,63.3,58,335,4.34,4.35,2.75,2,1,2


In [107]:
data = data.drop(data.columns[[1,2,3]], axis=1) 

In [108]:
# data.isnull().sum()
# data = data.iloc[:10]
# # print("hah ", data.iloc[1])
# for row in data:
#     print (row)
#     for val in row:
#         val = int(val)
# data.dtypes
data = data.apply(pd.to_numeric) 
data.dtypes

carat          float64
depth          float64
table          float64
price            int64
x              float64
y              float64
z              float64
map_cut          int64
map_color        int64
map_clarity      int64
dtype: object

In [109]:
data["table"] = data['table'].astype(float) 
data.dtypes

carat          float64
depth          float64
table          float64
price            int64
x              float64
y              float64
z              float64
map_cut          int64
map_color        int64
map_clarity      int64
dtype: object

In [110]:
data.head(10)

,carat,depth,table,price,x,y,z,map_cut,map_color,map_clarity
1,0.23,61.5,55.0,326,3.95,3.98,2.43,5,6,2
2,0.21,59.8,61.0,326,3.89,3.84,2.31,4,6,3
3,0.23,56.9,65.0,327,4.05,4.07,2.31,2,6,5
4,0.29,62.4,58.0,334,4.20,4.23,2.63,4,2,4
5,0.31,63.3,58.0,335,4.34,4.35,2.75,2,1,2
6,0.24,62.8,57.0,336,3.94,3.96,2.48,3,1,6
7,0.24,62.3,57.0,336,3.95,3.98,2.47,3,2,7
8,0.26,61.9,55.0,337,4.07,4.11,2.53,3,3,3
9,0.22,65.1,61.0,337,3.87,3.78,2.49,1,6,4
10,0.23,59.4,61.0,338,4.00,4.05,2.39,3,3,5


In [111]:
#normalize
def normalize_data(col):
#     print ("col : ", col)
    max_val = col.max()
#     print ("mmax : ", max_val)
    min_val = col.min()
#     print("first _val : ", col[0])
    for i in range(len(col)):
#         print("val : ", col[i])
        col[i] = (col[i]-min_val)/(max_val-min_val)
#         print("afterval : ", col[i])
    return col
#     result = data.copy()
#     for feature_col in col:
#         max_value = data[feature_col].max()
#         min_value = data.columns[[feature_col]].min()
#         result.columns[[feature_col]] = (data.columns[[feature_col]] - min_value) / (max_value - min_value)
#     return result

In [112]:
col_to_normalize = ["carat","depth","table", "x","y", "z"]
for col in col_to_normalize:
    data[col] = normalize_data(data[col].values)
# result = normalize_data(col_to_normalize)

In [113]:
data.head(15)

,carat,depth,table,price,x,y,z,map_cut,map_color,map_clarity
1,0.006237,0.513889,0.230769,326,0.367784,0.067572,0.076415,5,6,2
2,0.002079,0.466667,0.346154,326,0.362197,0.065195,0.072642,4,6,3
3,0.006237,0.386111,0.423077,327,0.377095,0.069100,0.072642,2,6,5
4,0.018711,0.538889,0.288462,334,0.391061,0.071817,0.082704,4,2,4
5,0.022869,0.563889,0.288462,335,0.404097,0.073854,0.086478,2,1,2
6,0.008316,0.550000,0.269231,336,0.366853,0.067233,0.077987,3,1,6
7,0.008316,0.536111,0.269231,336,0.367784,0.067572,0.077673,3,2,7
8,0.012474,0.525000,0.230769,337,0.378957,0.069779,0.079560,3,3,3
9,0.004158,0.613889,0.346154,337,0.360335,0.064177,0.078302,1,6,4
10,0.006237,0.455556,0.346154,338,0.372439,0.068761,0.075157,3,3,5


In [114]:
def euclidean_distance(row1, row2):
    print ("row1 :", row1)
    print ("row2 :", row2)
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

In [115]:
# Separate X and y data
Y = data.price
X = data.drop('price', axis=1)
# Y

1         326
2         326
3         327
4         334
5         335
         ... 
53936    2757
53937    2757
53938    2757
53939    2757
53940    2757
Name: price, Length: 53940, dtype: int64

In [116]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25,random_state=1)
# print(X_train)
# print(Y_train)

          carat     depth     table         x         y         z  map_cut  \
41945  0.031185  0.538889  0.288462  0.416201  0.077080  0.088365        4   
21480  0.274428  0.552778  0.307692  0.680633  0.123260  0.144025        4   
4452   0.145530  0.525000  0.250000  0.575419  0.105772  0.120755        5   
24949  0.311850  0.530556  0.269231  0.708566  0.128183  0.148113        4   
7282   0.166320  0.500000  0.365385  0.596834  0.107640  0.122327        4   
...         ...       ...       ...       ...       ...       ...      ...   
50058  0.103950  0.550000  0.307692  0.517691  0.095246  0.110377        3   
32512  0.041580  0.580556  0.269231  0.438547  0.078947  0.094025        2   
5193   0.147609  0.547222  0.384615  0.563315  0.101868  0.118868        3   
12173  0.203742  0.519444  0.250000  0.632216  0.115789  0.132075        5   
33004  0.024948  0.497222  0.250000  0.410615  0.075212  0.084591        5   

       map_color  map_clarity  
41945          3            3  

In [117]:
# Changing the index of the records to sequential
X_train.index=range(len(X_train))
Y_train.index=range(len(X_train))
X_test.index=range(len(X_test))
Y_test.index=range(len(Y_test))

In [118]:
Y_train

0          504
1         9576
2         3619
3        13417
4         4200
         ...  
40450     2202
40451      798
40452     3776
40453     5182
40454      813
Name: price, Length: 40455, dtype: int64

In [119]:
# Function to return the list of distances of the test records from train records
def distNeighbours(X_train,Y_train,X_test,K):
    distance=[]
    for i in range(len(X_train)):
        eDistance=0
        for j in range(len(X_train.columns)):   
            eDistance+=round(np.sqrt(pow((X_train.iloc[i,j]-X_test[j]),2)),2)
        distance.append((eDistance,i,Y_train.iloc[i]))
        distance=sorted(distance, key=lambda x: x[0])[0:K]
    return distance

# Predict the output of the numeric variables based on K nearest neighbours
# Output is the mean of the K nearest neighbours
def predictOutputNumeric(X_train,Y_train,X_test,K):
    neighbours=[]
    responses=[]
    for i in range(len(X_test)):
        neighbours.append(distNeighbours(X_train,Y_train,X_test.iloc[i,:],K))
    for i in neighbours:
        mean=0
        for j in i:
            mean+=j[-1]
        mean=mean/K
        responses.append(mean)
    return responses

In [120]:
# Predict GPA
output=predictOutputNumeric(X_train,Y_train,X_test,4)

KeyboardInterrupt: 

In [ ]:
print ("output  : ", output)

In [90]:
print ("output  : ", output)

output  :  [331.5, 333.25, 335.5]


In [91]:
# Accuarcy of the numerical predictions
def getAccuracyNumeric(actual,predicted):
    error=0
    for i in range(len(predicted)):
        error+=pow((actual[i]-predicted[i]),2)
    error=error/len(predicted)-1
    return 100-error

In [92]:
print('Accuracy from the code: {:^0.2f}'.format(getAccuracyNumeric(Y_test,output),2))

Accuracy from the code: 86.65


In [ ]:
print (output[0]-Y_test[0])